In [2]:
import ray

ray.init(num_cpus = 4)
@ray.remote
class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1
        return self.value

# Create ten Counter actors.
counters = [Counter.remote() for _ in range(10)]

# Increment each Counter once and get the results. These tasks all happen in
# parallel.
results = ray.get([c.increment.remote() for c in counters])
print(results) 

# Increment the first Counter five times. These tasks are executed serially and share state.
results = ray.get([counters[0].increment.remote() for _ in range(5)])
print(results)  # prints [2, 3, 4, 5, 6]

2021-06-16 18:58:20,101	INFO services.py:1317 -- View the Ray dashboard at http://127.0.0.1:8265
2021-06-16 18:58:24,691	WARNING worker.py:1121 -- WARNING: 12 PYTHON workers have been started on a node of the id: 3cff0c16568829b1666a7fe9ad5149ec5d651d6fe5630fc05f18c5c5 and address: 192.168.0.139. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2, 3, 4, 5, 6]


In [1]:
class Counter(object):
    def __init__(self):
        self.value = 0
   
    def increment(self):
        self.value += 1
        return self.value

from dask.distributed import Client

client = Client()

# Create 10 Actors, and call increment() once on each of them
counters = [client.submit(Counter, actor=True).result() for _ in range(10)]
results = [c.increment().result() for c in counters]
print(results)

# Call increment() 5 times on the first Actor in the list
results = [counters[0].increment().result() for _ in range(5)]
print(results)

/Users/smanthony/opt/anaconda3/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60801 instead
  http_address["port"], self.http_server.port


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2, 3, 4, 5, 6]


In [ ]:
class Counter(object):
    def __init__(self):
        self.value = 0
   
    def increment(self):
        self.value += 1
        return self.value


from dask.distributed import Client

client = Client()
workers = []
for _ in range(10):
    future = client.submit(Counter, actor=True)  # Create a Counter on a worker
    counter = future.result()                    # Get back a pointer to that object
    future = counter.increment()
    result = future.result() # Call remote method
    workers.append(result)
print("10 workers", workers)

result = []
for _ in range(5):
    future = counter.increment()
    values = future.result()
    result.append(values)
print("Here is the list after 5 increments\n", result)